In [1]:
from pyleaves.utils.callback_utils import BackupAndRestore

import arrow
import copy
from datetime import datetime, timedelta
import numpy as np
import neptune
import os
import random
from stuf import stuf
from pathlib import Path
os.environ["CUDA_VISIBLE_DEVICES"] = '6'
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except:
    print('setting memory growth failed, continuing anyway.')
random.seed(84)
np.random.seed(58)
tf.random.set_seed(34)
from tensorflow.keras import backend as K
from pyleaves.mains.baseline_train_pipeline import build_model, build_or_restore_model, create_dataset, load_data, prep_dataset, resize_image, ImageLoggerCallback, EarlyStopping, neptune_logger
from pyleaves.datasets import leaves_dataset, fossil_dataset, pnas_dataset, base_dataset
import matplotlib.pyplot as plt
from pprint import pprint
##################################################
##################################################
def plot_batch(x, y=None, idx = None, encoder=None, rows=4, cols=2, figsize=(15,15)):
    assert rows*cols >= len(x), print('Not enough requested rows or columns to plot all images')
    
    x_norm = (x - x.numpy().min())/(x.numpy().max() - x.numpy().min())
    
    if encoder is not None:
        y = [encoder[i] for i in y.numpy()]
    if idx is not None:
        idx = idx.numpy()
        y = ['-'.join([str(idx[i]),y[i]]) for i in range(idx.shape[0])]
    
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.ravel()
    for i in range(x_norm.shape[0]): #len(axes)):
        axes[i].imshow(x_norm[i,...])
        if y is not None:
            axes[i].set_title(y[i])
            
    return fig, axes

date_format = '%Y-%m-%d_%H-%M-%S'

def initialize_experiment(PARAMS, experiment_start_time=None):
    PARAMS['experiment_name'] = '_'.join([PARAMS['dataset_name'], PARAMS['model_name']])
    PARAMS['experiment_dir'] = os.path.join(PARAMS['neptune_experiment_dir'], PARAMS['experiment_name'])

    PARAMS['experiment_start_time'] = experiment_start_time or datetime.now().strftime(date_format)
    PARAMS['log_dir'] = os.path.join(PARAMS['experiment_dir'], 'log_dir__'+PARAMS['experiment_start_time'])
    PARAMS['model_dir'] = os.path.join(PARAMS['log_dir'],'model_dir')
    PARAMS['saved_model_path'] = str(Path(PARAMS['model_dir']) / Path('saved_model'))
    PARAMS['checkpoints_path'] = str(Path(PARAMS['model_dir']) / Path('checkpoints'))
    
def restore_or_initialize_experiment(PARAMS, restore_last=False, prefix='log_dir__', verbose=0):
#     date_format = '%Y-%m-%d_%H-%M-%S'
    PARAMS = copy.deepcopy(PARAMS)
    PARAMS['experiment_name'] = '_'.join([PARAMS['dataset_name'], PARAMS['model_name']])
    PARAMS['experiment_dir'] = os.path.join(PARAMS['neptune_experiment_dir'], PARAMS['experiment_name'])

    if restore_last:
        experiment_files = [(exp_name.split(prefix)[-1], exp_name) for exp_name in os.listdir(PARAMS['experiment_dir'])]
        keep_files = []
        for i in range(len(experiment_files)):
            exp = experiment_files[i]
            try:
                keep_files.append((datetime.strptime(exp[0], date_format), exp[1]))
                if verbose >= 1: print(f'Found previous experiment {exp[1]}')
            except ValueError:
                if verbose >=2: print(f'skipping invalid file {exp[1]}')
                pass
        
        experiment_files = sorted(keep_files, key= lambda exp: exp[0])
        if type(experiment_files)==list and len(experiment_files)>0:
            experiment_file = experiment_files[-1]
            PARAMS['experiment_start_time'] = experiment_file[0].strftime(date_format)
            initialize_experiment(PARAMS, experiment_start_time=PARAMS['experiment_start_time'])
            if verbose >= 1: print(f'Continuing experiment with start time =', PARAMS['experiment_start_time'])
            return PARAMS
        else:
            print('No previous experiment in',PARAMS['experiment_dir'], 'with prefix',prefix)

    PARAMS['experiment_start_time'] = datetime.now().strftime(date_format)
    initialize_experiment(PARAMS, experiment_start_time=PARAMS['experiment_start_time'])
    if verbose >= 1: print('Initializing new experiment at time:', PARAMS['experiment_start_time'] )
    return PARAMS
    
##################################################
##################################################
##################################################

##################################################
##################################################
##################################################
    
    
PARAMS = {'neptune_project_name':'jacobarose/sandbox',
          'neptune_experiment_dir':'/media/data/jacob/sandbox_logs',
          'optimizer':'Adam',
          'loss':'categorical_crossentropy',
          'lr':1e-5,
          'color_mode':'grayscale',
          'num_channels':3,
          'BATCH_SIZE':8,
          'buffer_size':400,
          'num_epochs':150,
          'dataset_name':'Fossil',#'Leaves',#'PNAS',
          'threshold':2,
          'frozen_layers':None,
          'model_name':'vgg16',#'resnet_50_v2',
          'splits':{'train':0.5,'validation':0.5},
          'seed':45}

PARAMS = stuf(PARAMS)

PARAMS['exclude_classes'] = ['notcataloged','notcatalogued', 'II. IDs, families uncertain', 'Unidentified']
PARAMS['regularization'] = {'l1':3e-4}
PARAMS['METRICS'] = ['accuracy','precision','recall']
PARAMS['target_size'] = (256,256)#(512,512)#(768,768)#(128,128)#
PARAMS['augmentations'] = [{'flip':1.0}]

##################################################
##################################################
##################################################

##################################################
##################################################
##################################################


PARAMS = restore_or_initialize_experiment(PARAMS, restore_last=True, prefix='log_dir__', verbose=2)
pprint(PARAMS)

##################################################
##################################################
from pyleaves.utils import ensure_dir_exists

neptune.init(project_qualified_name=PARAMS['neptune_project_name'])
neptune.create_experiment(name=PARAMS['experiment_name']+'-'+str(dict(PARAMS['splits'])), params=PARAMS)


ensure_dir_exists(PARAMS['log_dir'])
ensure_dir_exists(PARAMS['model_dir'])
neptune.append_tag(PARAMS['dataset_name'])
neptune.append_tag(PARAMS['model_name'])
neptune.append_tag(str(PARAMS['target_size']))
neptune.append_tag(PARAMS['num_channels'])
neptune.append_tag(PARAMS['color_mode'])
K.clear_session()
tf.random.set_seed(PARAMS['seed'])

##################################################
##################################################



# PARAMS['experiment_name'] = '_'.join([PARAMS['dataset_name'], PARAMS['model_name']])
# PARAMS['experiment_dir'] = os.path.join(PARAMS['neptune_experiment_dir'], PARAMS['experiment_name'])
# PARAMS['log_dir'] = os.path.join(PARAMS['experiment_dir'], 'log_dir__'+PARAMS['experiment_start_time'])
# PARAMS['model_dir'] = os.path.join(PARAMS['log_dir'],'model_dir')
# from pathlib import Path
# PARAMS['saved_model_path'] = str(Path(PARAMS['model_dir']) / Path('saved_model'))
# PARAMS['checkpoints_path'] = str(Path(PARAMS['model_dir']) / Path('checkpoints'))
# PARAMS['checkpoints_path'] = str(Path(PARAMS['checkpoints_dir']) / Path(PARAMS['model_name']+'.ckpt-{epoch}'))
# pprint(PARAMS)

# restore_or_initialize_experiment(PARAMS, restore_last=True, prefix='log_dir_categorical_crossentropy_')#False)



train_dataset, validation_dataset, data_files, excluded = create_dataset(dataset_name=PARAMS['dataset_name'],
                                                               threshold=PARAMS['threshold'],
                                                               batch_size=PARAMS['BATCH_SIZE'],
                                                               buffer_size=PARAMS['buffer_size'],
                                                               exclude_classes=PARAMS['exclude_classes'],
                                                               target_size=PARAMS['target_size'],
                                                               num_channels=PARAMS['num_channels'],
                                                               color_mode=PARAMS['color_mode'],
                                                               splits=PARAMS['splits'],
                                                               augmentations=PARAMS['augmentations'],
                                                               seed=PARAMS['seed'])

PARAMS['num_classes'] = data_files.num_classes
PARAMS['splits_size'] = {'train':{},
                   'validation':{}}
PARAMS['splits_size']['train'] = int(data_files.num_samples*PARAMS['splits']['train'])
PARAMS['splits_size']['validation'] = int(data_files.num_samples*PARAMS['splits']['validation'])

PARAMS['steps_per_epoch'] = PARAMS['splits_size']['train']//PARAMS['BATCH_SIZE']
PARAMS['validation_steps'] = PARAMS['splits_size']['validation']//PARAMS['BATCH_SIZE']

neptune.set_property('num_classes',PARAMS['num_classes'])
neptune.set_property('steps_per_epoch',PARAMS['steps_per_epoch'])
neptune.set_property('validation_steps',PARAMS['validation_steps'])

# TODO: log encoder contents as dict
encoder = base_dataset.LabelEncoder(data_files.classes)

PARAMS['base_learning_rate'] = PARAMS['lr']
PARAMS['input_shape'] = (*PARAMS['target_size'],PARAMS['num_channels'])


##################################################
##################################################
# print(len(list(set(data_files.classes))))
# print(len(list(set(data_files.classes) - {'notcataloged','notcatalogued'})))
# excluded.data

/media/data/conda/jacob/envs/pyleaves2/lib/python3.7/site-packages/scikitplot/plotters.py:37: DeprecationWarning: This module was deprecated in version 0.3.0 and its functions are spread throughout different modules. Please check the documentation and update your function calls as soon as possible. This module will be removed in 0.4.0
  DeprecationWarning)


skipping invalid file log_dir
Found previous experiment log_dir__2020-07-23_07-55-50
Continuing experiment with start time = 2020-07-23_07-55-50
{'BATCH_SIZE': 8,
 'METRICS': ['accuracy', 'precision', 'recall'],
 'augmentations': [{'flip': 1.0}],
 'buffer_size': 400,
 'checkpoints_path': '/media/data/jacob/sandbox_logs/Fossil_vgg16/log_dir__2020-07-23_07-55-50/model_dir/checkpoints',
 'color_mode': 'grayscale',
 'dataset_name': 'Fossil',
 'exclude_classes': ['notcataloged',
                     'notcatalogued',
                     'II. IDs, families uncertain',
                     'Unidentified'],
 'experiment_dir': '/media/data/jacob/sandbox_logs/Fossil_vgg16',
 'experiment_name': 'Fossil_vgg16',
 'experiment_start_time': '2020-07-23_07-55-50',
 'frozen_layers': None,
 'log_dir': '/media/data/jacob/sandbox_logs/Fossil_vgg16/log_dir__2020-07-23_07-55-50',
 'loss': 'categorical_crossentropy',
 'lr': 1e-05,
 'model_dir': '/media/data/jacob/sandbox_logs/Fossil_vgg16/log_dir__2020-07-23_

https://ui.neptune.ai/jacobarose/sandbox/e/SAN-580


In [7]:
# encoder.classes
data_files
type(excluded)
excluded.__repr__()


'Fossil:\n    num_samples: 2435\n    num_classes: 2\n    class_count_threshold: 2\n        '

In [1]:
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import random
import os
from pprint import pprint
os.environ["CUDA_VISIBLE_DEVICES"] = '5'
import tensorflow as tf
random.seed(84)
np.random.seed(58)
tf.random.set_seed(34)
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, TensorBoard, LearningRateScheduler, EarlyStopping
from tensorflow.keras import backend as K
import tensorflow_datasets as tfds
from pyleaves.models import resnet, vgg16
from pyleaves.datasets import leaves_dataset, fossil_dataset, pnas_dataset, base_dataset
import neptune
import arrow
from pyleaves.utils import ensure_dir_exists
from more_itertools import unzip

##########################################################################
##########################################################################

def image_reshape(x):
    return [
        tf.image.resize(x, (7, 7)),
        tf.image.resize(x, (14, 14)),
        x
    ]


def load_img(image_path):#, img_size=(224,224)):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def resize(image, h=512, w=512):
    return tf.image.resize_with_pad(image, target_height=h, target_width=w)


def rgb2gray_3channel(img, label):
    '''
    Convert rgb image to grayscale, but keep num_channels=3
    '''
    img = tf.image.rgb_to_grayscale(img)
    img = tf.image.grayscale_to_rgb(img)
    return img, label

def rgb2gray_1channel(img, label):
    '''
    Convert rgb image to grayscale, num_channels from 3 to 1
    '''
    img = tf.image.rgb_to_grayscale(img)
    return img, label


def _cond_apply(x, y, func, prob):
    """Conditionally apply func to x and y with probability prob.
    """
    return tf.cond((tf.random.uniform([], 0, 1) >= (1.0 - prob)), lambda: func(x,y), lambda: (x,y))


def augment_sample(x, y):
    x = tf.image.random_flip_left_right(x, seed=2)
    x = tf.image.random_flip_up_down(x, seed=2)
    return x, y
                

preprocess_input(tf.zeros([4, 224, 224, 3]))
def apply_preprocess(x, y, num_classes=10):
    return preprocess_input(x), tf.one_hot(y, depth=num_classes)
    
    

def create_mnist_dataset(batch_size):
    (train_images, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
    train_images = train_images.reshape([-1, 28, 28, 1]).astype('float32')
    train_images = train_images/127.5  - 1
    dataset = tf.data.Dataset.from_tensor_slices(train_images)
    dataset = dataset.map(image_reshape)
    dataset = dataset.cache()
    dataset = dataset.shuffle(len(train_images))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(1)
    return dataset


def prep_dataset(dataset,
                 batch_size=32, 
                 buffer_size=100, 
                 shuffle=False,
                 target_size=(512,512),
                 num_channels=3,
                 color_mode='grayscale',
                 num_classes=10,
                 augment=False, 
                 aug_prob=1.0):
    dataset = dataset.map(lambda x,y: (resize(x, *target_size),y), num_parallel_calls=-1)
    
    dataset = dataset.map(lambda x,y: apply_preprocess(x,y,num_classes),num_parallel_calls=-1)
    
    if color_mode=='grayscale':
        if num_channels==3:
            dataset = dataset.map(lambda x,y: rgb2gray_3channel(x, y), num_parallel_calls=-1)
        elif num_channels==1:
            dataset = dataset.map(lambda x,y: rgb2gray_1channel(x, y), num_parallel_calls=-1)
    
    dataset = dataset.cache()
    dataset = dataset.repeat()
    
    if augment:
        dataset = dataset.map(lambda x,y: _cond_apply(x, y, augment_sample, prob=aug_prob), num_parallel_calls=-1)
#         dataset = dataset.map(augment_sample, num_parallel_calls=-1)
    if shuffle:
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(1)
    return dataset


def create_Imagenette_dataset(batch_size,
                              target_size=(512,512), 
                              augment_train=True,
                              aug_prob=1.0):
    
    data, info = tfds.load('Imagenette', as_supervised=True, with_info=True)
    train_data = prep_dataset(data['train'], 
                              batch_size=batch_size, 
                              buffer_size=info.splits['train'].num_examples,
                              shuffle=True,
                              target_size=target_size,
                              augment=augment_train,
                              aug_prob=aug_prob)
    val_data = prep_dataset(data['validation'], 
                            batch_size=batch_size,
                            target_size=target_size)
    
    return train_data, val_data, info



def partition_data(data, partitions=OrderedDict({'train':0.5,'test':0.5})):
    '''
    Split data into named partitions by fraction

    Example:
    --------
    >> split_data = partition_data(data, partitions=OrderedDict({'train':0.4,'val':0.1,'test':0.5}))
    '''
    num_rows = len(data)
    output={}
    taken = 0.0
    for k,v in partitions.items():
        idx = (int(taken*num_rows),int((taken+v)*num_rows))
        output.update({k:data[idx[0]:idx[1]]})
        taken+=v
    assert taken <= 1.0
    return output

def load_data(dataset_name='PNAS', splits={'train':0.7,'validation':0.3}, threshold=50):
    datasets = {
            'PNAS': pnas_dataset.PNASDataset(),
            'Leaves': leaves_dataset.LeavesDataset(),
            'Fossil': fossil_dataset.FossilDataset()
            }
    data_files = datasets[dataset_name]
    
    data_files.exclude_rare_classes(threshold=threshold)
    encoder = base_dataset.LabelEncoder(data_files.classes)
    data_files, _ = data_files.enforce_class_whitelist(class_names=encoder.classes)

    x = list(data_files.data['path'].values)
    y = np.array(encoder.encode(data_files.data['family']))

    shuffled_data = list(zip(x,y))
    random.shuffle(shuffled_data)
    partitioned_data = partition_data(data=shuffled_data,
                                      partitions=OrderedDict(splits))
    split_data = {k:v for k,v in partitioned_data.items() if len(v)>0}
    
    for subset, subset_data in split_data.items():
        split_data[subset] = [list(i) for i in unzip(subset_data)]
    
    paths = tf.data.Dataset.from_tensor_slices(split_data['train'][0])
    labels = tf.data.Dataset.from_tensor_slices(split_data['train'][1])
    train_data = tf.data.Dataset.zip((paths, labels))
    train_data = train_data.map(lambda x,y: (tf.image.convert_image_dtype(load_img(x)*255.0,dtype=tf.uint8),y), num_parallel_calls=-1)
    
    paths = tf.data.Dataset.from_tensor_slices(split_data['validation'][0])
    labels = tf.data.Dataset.from_tensor_slices(split_data['validation'][1])
    validation_data = tf.data.Dataset.zip((paths, labels))
    validation_data = validation_data.map(lambda x,y: (tf.image.convert_image_dtype(load_img(x)*255.0,dtype=tf.uint8),y), num_parallel_calls=-1)
    
    return {'train':train_data, 
            'validation':validation_data}, data_files


def create_dataset(dataset_name='PNAS',
                   batch_size=32,
                   target_size=(512,512),
                   num_channels=1,
                   color_mode='grayscale',
                   splits={'train':0.7,'validation':0.3},
                   augment_train=True,
                   aug_prob=1.0):
    
    dataset, data_files = load_data(dataset_name=dataset_name, splits=splits)
    train_data = prep_dataset(dataset['train'], 
                              batch_size=batch_size, 
                              buffer_size=int(data_files.num_samples*splits['train']),
                              shuffle=True,
                              target_size=target_size,
                              num_channels=num_channels,
                              color_mode=color_mode,
                              num_classes=data_files.num_classes,
                              augment=augment_train,
                              aug_prob=aug_prob)
    val_data = prep_dataset(dataset['validation'], 
                            batch_size=batch_size,
                            target_size=target_size,
                            num_channels=num_channels,
                            color_mode=color_mode,
                            num_classes=data_files.num_classes)
    return train_data, val_data, data_files


# def create_pnas_dataset(batch_size):
#     data_files = pnas_dataset.PNASDataset()
#     dataset = tf.data.Dataset.from_tensor_slices(data_files.data['path'])
#     dataset = dataset.map(load_img, num_parallel_calls=-1)
#     dataset = dataset.map(resize, num_parallel_calls=-1)
#     dataset = dataset.cache()
#     dataset = dataset.shuffle(20)#len(data_files.data))
#     dataset = dataset.batch(batch_size, drop_remainder=True)
#     dataset = dataset.prefetch(1)
#     return dataset

##########################################################################
##########################################################################
    

def build_base_vgg16_RGB(PARAMS):
#     if PARAMS['optimizer']=='Adam':
#         optimizer = tf.keras.optimizers.Adam(learning_rate=PARAMS['lr'])
    
    base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
                                             include_top=False,
                                             input_tensor=Input(shape=(*PARAMS['target_size'],3)))

    return base


def build_head(base, num_classes=10):
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    dense1 = tf.keras.layers.Dense(2048,activation='relu',name='dense1')
    dense2 = tf.keras.layers.Dense(512,activation='relu',name='dense2')
    prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
    model = tf.keras.Sequential([
        base,
        global_average_layer,dense1,dense2,
        prediction_layer
        ])
    return model

from functools import partial

def build_model(PARAMS):
    '''
    model_params = {
                'num_classes':PARAMS['num_classes'],
                'frozen_layers':PARAMS['frozen_layers'],
                'input_shape':(*PARAMS['target_size'],PARAMS['num_channels']),
                'base_learning_rate':PARAMS['lr'],
                'regularization':PARAMS['regularization'],
                'loss':'categorical_crossentropy'.
                'METRICS':['accuracy']
                }
    '''
    
    if PARAMS['model_name']=='vgg16':
        if PARAMS['num_channels']==1:
            model_builder = vgg16.VGG16GrayScale(PARAMS)
            build_base = model_builder.build_base
        else:
            build_base = partial(build_base_vgg16_RGB, PARAMS=PARAMS)
            
    elif PARAMS['model_name'].startswith('resnet'):
        model_builder = resnet.ResNet(PARAMS)
        build_base = model_builder.build_base

    base = build_base()
    model = build_head(base, num_classes=PARAMS['num_classes'])

    if PARAMS['optimizer']=='Adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=PARAMS['lr'])
        
    if PARAMS['loss']=='categorical_crossentropy':
        loss = 'categorical_crossentropy'
        
    if 'accuracy' in PARAMS['METRICS']:
        METRICS = ['accuracy']
    
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=METRICS)

    return model





def log_data(logs):
    for k, v in logs.items():
        neptune.log_metric(k, v)

        
##########################################################################
##########################################################################

        
def plot_sample(sample, num_res=1):
    num_samples = min(64, len(sample[0]))

    grid = gridspec.GridSpec(num_res, num_samples)
    grid.update(left=0, bottom=0, top=1, right=1, wspace=0.01, hspace=0.01)
    fig = plt.figure(figsize=[num_samples, num_res])
    for x in range(num_res):
        images = sample[x].numpy() #this converts the tensor to a numpy array
        images = np.squeeze(images)
        for y in range(num_samples):
            ax = fig.add_subplot(grid[x, y])
            ax.set_axis_off()
            ax.imshow((images[y] + 1.0)/2, cmap='gray')
    plt.show()
    

# neptune_logger = tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: log_data(logs))
neptune_logger = tf.keras.callbacks.LambdaCallback(on_batch_end=lambda batch, logs: log_data(logs),
                                                  on_epoch_end=lambda epoch, logs: log_data(logs))

In [2]:
from pyleaves.utils.neptune_utils import ImageLoggerCallback

def train_imagenette(PARAMS):

    neptune.append_tag(PARAMS['dataset_name'])
    neptune.append_tag(PARAMS['model_name'])
    
    K.clear_session()
    tf.random.set_seed(34)
    target_size = PARAMS['target_size']
    BATCH_SIZE = PARAMS['BATCH_SIZE']
    
    train_dataset, validation_dataset, info = create_Imagenette_dataset(BATCH_SIZE, target_size=target_size, augment_train=PARAMS['augment_train'])
    num_classes = info.features['label'].num_classes
    
    encoder = base_dataset.LabelEncoder(info.features['label'].names)
    
    train_dataset = train_dataset.map(lambda x,y: apply_preprocess(x,y,num_classes),num_parallel_calls=-1)
    validation_dataset = validation_dataset.map(lambda x,y: apply_preprocess(x,y,num_classes),num_parallel_calls=-1)
    
    PARAMS['num_classes'] = num_classes
    steps_per_epoch = info.splits['train'].num_examples//BATCH_SIZE
    validation_steps = info.splits['validation'].num_examples//BATCH_SIZE

    neptune.set_property('num_classes',num_classes)
    neptune.set_property('steps_per_epoch',steps_per_epoch)
    neptune.set_property('validation_steps',validation_steps)


    optimizer = tf.keras.optimizers.Adam(learning_rate=PARAMS['learning_rate'])
    loss = 'categorical_crossentropy'
    METRICS = ['accuracy']

    base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
                                             include_top=False,
                                             input_tensor=Input(shape=(*target_size,3)))

    # TODO try freezing weights for input_shape != (224,224)

    model = build_head(base, num_classes=num_classes)

    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=METRICS)

    callbacks = [neptune_logger,
                 ImageLoggerCallback(data=train_dataset, freq=10, max_images=-1, name='train', encoder=encoder),
                 ImageLoggerCallback(data=validation_dataset, freq=10, max_images=-1, name='val', encoder=encoder),
                 EarlyStopping(monitor='val_loss', patience=2, verbose=1)]

    model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))
    pprint(PARAMS)
    history = model.fit(train_dataset,
                        epochs=10,
                        callbacks=callbacks,
                        validation_data=validation_dataset,
                        shuffle=True,
                        initial_epoch=0,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps)
    
    
def train_pnas(PARAMS):
    ensure_dir_exists(PARAMS['log_dir'])
    ensure_dir_exists(PARAMS['model_dir'])
    neptune.append_tag(PARAMS['dataset_name'])
    neptune.append_tag(PARAMS['model_name'])
    neptune.append_tag(str(PARAMS['target_size']))
    neptune.append_tag(PARAMS['num_channels'])
    neptune.append_tag(PARAMS['color_mode'])
    
    K.clear_session()
    tf.random.set_seed(34)
    
    
    train_dataset, validation_dataset, data_files = create_dataset(dataset_name=PARAMS['dataset_name'],
                                                                   batch_size=PARAMS['BATCH_SIZE'],
                                                                   target_size=PARAMS['target_size'],
                                                                   num_channels=PARAMS['num_channels'],
                                                                   color_mode=PARAMS['color_mode'],
                                                                   splits=PARAMS['splits'],
                                                                   augment_train=PARAMS['augment_train'],
                                                                   aug_prob=PARAMS['aug_prob'])
    
    
    PARAMS['num_classes'] = data_files.num_classes
    PARAMS['splits_size'] = {'train':{},
                       'validation':{}}
    PARAMS['splits_size']['train'] = data_files.num_samples*PARAMS['splits']['train']
    PARAMS['splits_size']['validation'] = data_files.num_samples*PARAMS['splits']['validation']

    steps_per_epoch = PARAMS['splits_size']['train']//PARAMS['BATCH_SIZE']
    validation_steps = PARAMS['splits_size']['validation']//PARAMS['BATCH_SIZE']
   
    neptune.set_property('num_classes',PARAMS['num_classes'])
    neptune.set_property('steps_per_epoch',steps_per_epoch)
    neptune.set_property('validation_steps',validation_steps)
    
    
    encoder = base_dataset.LabelEncoder(data_files.classes)
#     train_dataset = train_dataset.map(lambda x,y: apply_preprocess(x,y,PARAMS['num_classes']),num_parallel_calls=-1)
#     validation_dataset = validation_dataset.map(lambda x,y: apply_preprocess(x,y,PARAMS['num_classes']),num_parallel_calls=-1)

    
#     METRICS = ['accuracy']
    callbacks = [neptune_logger,
                 ImageLoggerCallback(data=train_dataset, freq=10, max_images=-1, name='train', encoder=encoder),
                 ImageLoggerCallback(data=validation_dataset, freq=10, max_images=-1, name='val', encoder=encoder),
                 EarlyStopping(monitor='val_loss', patience=30, verbose=1)]
    
    PARAMS['base_learning_rate'] = PARAMS['lr']
    PARAMS['input_shape'] = (*PARAMS['target_size'],PARAMS['num_channels'])
    model = build_model(PARAMS)
    
    
#     if PARAMS['optimizer']=='Adam':
#         optimizer = tf.keras.optimizers.Adam(learning_rate=PARAMS['lr'])
    
#     base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
#                                              include_top=False,
#                                              input_tensor=Input(shape=(*PARAMS['target_size'],3)))

#     model = build_head(base, num_classes=PARAMS['num_classes'])

#     model.compile(optimizer=optimizer,
#                   loss=PARAMS['loss'],
#                   metrics=METRICS)
    
    model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))
    pprint(PARAMS)
    history = model.fit(train_dataset,
                        epochs=PARAMS['num_epochs'],
                        callbacks=callbacks,
                        validation_data=validation_dataset,
                        shuffle=True,
                        initial_epoch=0,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps)
    
    
    for k,v in PARAMS.items():
        neptune.set_property(str(k),str(v))
    
    return history

In [3]:
# PARAMS = {'neptune_project_name':'jacobarose/sandbox',
#           'experiment_name':'pnas_test',
#           'experiment_dir':'/media/data/jacob/sandbox_logs',
#           'experiment_start_time':arrow.utcnow().format('YYYY-MM-DD_HH-mm-ss'),
#           'optimizer':'Adam',
#           'loss':'categorical_crossentropy',
#           'lr':1e-5,
#           'target_size':(224,224), #(256,256),
#           'BATCH_SIZE':48,
#           'num_epochs':150,
#           'dataset_name':'pnas',
#           'model_name':'vgg16',
#           'augment_train':True,
#           'splits':{'train':0.5,'validation':0.5}}

# PARAMS = {'neptune_project_name':'jacobarose/sandbox',
#           'experiment_name':'fossil_test',
#           'experiment_dir':'/media/data/jacob/sandbox_logs',
#           'experiment_start_time':arrow.utcnow().format('YYYY-MM-DD_HH-mm-ss'),
#           'optimizer':'Adam',
#           'loss':'categorical_crossentropy',
#           'lr':1e-5,
#           'METRICS':['accuracy'],
#           'target_size':(224,224),
#           'num_channels':1,
#           'BATCH_SIZE':48,
#           'num_epochs':150,
#           'dataset_name':'Fossil',
#           'model_name':'vgg16',
#           'augment_train':True,
#           'splits':{'train':0.5,'validation':0.5}}


from stuf import stuf


PARAMS = {'neptune_project_name':'jacobarose/sandbox',
          'experiment_dir':'/media/data/jacob/sandbox_logs',
          'experiment_start_time':arrow.utcnow().format('YYYY-MM-DD_HH-mm-ss'),
          'optimizer':'Adam',
          'loss':'categorical_crossentropy',
          'lr':1e-5,
#           'METRICS':'accuracy', #['accuracy'],
#           'target_size':(224,224),
          'color_mode':'grayscale',
          'num_channels':1,
          'BATCH_SIZE':48,
          'num_epochs':150,
          'dataset_name':'Fossil',
          'frozen_layers':None,
          'model_name':'vgg16',
          'augment_train':True,
          'aug_prob':0.5,
          'splits':stuf({'train':0.5,'validation':0.5})}

PARAMS = stuf(PARAMS)

PARAMS['experiment_name'] = '_'.join([PARAMS['dataset_name'], PARAMS['model_name']])
PARAMS['regularization'] = {'l1':1e-3}
PARAMS['METRICS'] = 'accuracy',
PARAMS['target_size'] = (224,224)


PARAMS['log_dir'] = os.path.join(PARAMS['experiment_dir'], PARAMS['experiment_name'], 'log_dir', PARAMS['loss'], PARAMS['experiment_start_time'])
PARAMS['model_dir'] = os.path.join(PARAMS['log_dir'],'model_dir')

neptune.init(project_qualified_name=PARAMS['neptune_project_name'])
with neptune.create_experiment(name=PARAMS['experiment_name']+'-'+str(PARAMS['splits']), params=PARAMS):
    train_pnas(PARAMS)

https://ui.neptune.ai/jacobarose/sandbox/e/SAN-463
INFO:tensorflow:Assets written to: /media/data/jacob/sandbox_logs/Fossil_vgg16/log_dir/categorical_crossentropy/2020-07-02_09-13-00/model_dir/vgg16_grayscale-saved_base_model/assets
Saved model vgg16_grayscale at path: /media/data/jacob/sandbox_logs/Fossil_vgg16/log_dir/categorical_crossentropy/2020-07-02_09-13-00/model_dir/vgg16_grayscale-saved_base_model
{'BATCH_SIZE': 48,
 'METRICS': ('accuracy',),
 'augment_train': True,
 'base_learning_rate': 1e-05,
 'color_mode': 'grayscale',
 'config_filepath': '/media/data/jacob/sandbox_logs/Fossil_vgg16/log_dir/categorical_crossentropy/2020-07-02_09-13-00/model_dir/vgg16_grayscale-model_config.json',
 'dataset_name': 'Fossil',
 'experiment_dir': '/media/data/jacob/sandbox_logs',
 'experiment_name': 'Fossil_vgg16',
 'experiment_start_time': '2020-07-02_09-13-00',
 'frozen_layers': None,
 'input_shape': (224, 224, 1),
 'log_dir': '/media/data/jacob/sandbox_logs/Fossil_vgg16/log_dir/categorical_c